In [ ]:
import sys
# Assuming we are in the notebook directory add this so that we can import the library
sys.path.append('..')

import numpy as np
import elfi
import elfi.examples.ma2 as elfi_examples
from functools import partial

import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')
%matplotlib inline

In [ ]:
n = 100
t1_0 = 0.6
t2_0 = 0.2

# Set up observed data y
latents = np.random.randn(n+2)
y = elfi_examples.MA2(n, t1_0, t2_0, n_sim=1, latents=latents)

# Plot
plt.figure(figsize=(11, 6));
plt.plot(np.arange(0,n),y[0,:]);
plt.scatter(np.arange(-2,n), latents);

In [ ]:
# Set up the simulator
simulator = partial(elfi_examples.MA2, n)

# Set up autocovariance summaries
ac1 = partial(elfi_examples.autocov, 1)
ac2 = partial(elfi_examples.autocov, 2)

# Specify the graphical model
t1 = elfi.Prior('t1', 'uniform', 0, 1)
t2 = elfi.Prior('t2', 'uniform', 0, 1)
Y = elfi.Simulator('MA2', simulator, t1, t2, observed=y)
S1 = elfi.Summary('S1', ac1, Y)
S2 = elfi.Summary('S2', ac2, Y)
d = elfi.Discrepancy('d', elfi_examples.distance, S1, S2, store=elfi.MemoryStore())

In [ ]:
# Specify the number of simulations and set up rejection sampling
rej = elfi.Rejection(d, [t1, t2], batch_size=100000)
N = 50000

In [ ]:
# Time and run the simulator in parallel
%time result = rej.sample(N, quantile=0.01)

In [ ]:
# Run again, but this time we are able to use the cached results
%time result = rej.sample(N, quantile=0.01)

In [ ]:
[t1_post, t2_post] = result['samples']
print("Number of accepted samples {} with threshold {:.2f}".format(len(t1_post), result['threshold']))
print("Posterior means: {:.2f} {:.2f}".format(t1_post.mean(), t2_post.mean()))

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(14,5));
ax[0].hist(t1_post, bins=20);
ax[0].set_title("Posterior for t1");
ax[1].hist(t2_post, bins=20);
ax[1].set_title("Posterior for t2");